# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

### Question:

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

### Answer:

This is a classification problem. The output takes on class labels (classification), not continuous values (regression).

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
n_students = len(student_data)
n_features = len(student_data.columns) - 1
n_passed = len([s for s in student_data['passed'] if s == 'yes'])
n_failed = n_students - n_passed
grad_rate = 100.0 * n_passed / n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [7]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn import svm
clf = svm.SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.008
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [8]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.006
F1 score for training set: 0.877272727273


In [9]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.851612903226


## Support Vector Machines

### Complexity

Computational: 

Training: O(max(n,d)  min(n,d)^2), where n is the number of points, and d is the number of dimensions [1]. 

Testing: depends on the kernel used.

Space: O(n^2) [2].

### General Applications

* Data with large numbers of features

* Text and hypertext categorization
* Image classification

### Strengths

* Effective when there are large numbers of features
* Decision function is memory-efficient

### Weaknesses

* Does not directly give probability estimates - they have to be computed seperately & expensively
* If # features >> # samples, SVNs will perform poorly
* More applicable to binary problems


### Why choose this model?

There are quite a few features in our data and not too many data points to support the potential complexity that any model will end up with to handle that many features - SVNs will perform acceptably in this case. I'm interested to see how it compares to other models. It is also a binary classification model - which is relevant to our data (multiclass SVM models also exist).

[1] Chapelle, Olivier. "Training a support vector machine in the primal." Neural Computation 19.5 (2007): 1155-1178.

[2] http://www.jmlr.org/papers/volume6/tsang05a/tsang05a.pdf

In [10]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
print "========="
print "===SVC==="
print "========="
train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf, X_train[0:100], y_train[0:100], X_test, y_test)
train_predict(clf, X_train[0:200], y_train[0:200], X_test, y_test)
train_predict(clf, X_train[0:300], y_train[0:300], X_test, y_test)

===SVC===
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.011
Predicting labels using SVC...
Done!
Prediction time (secs): 0.007
F1 score for training set: 0.877272727273
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.851612903226
------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.87898089172
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.853658536585
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.004
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.894915254237
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.826666666667
-

## Forest of Randomized Trees

### Complexity

Computational: 

Training: O(t*m*n*log n) where t is the number of decision trees, m is the number of features, and n is the number of data points.

Testing: O(t*log(n))

Space: O(t*2^m)

### General Applications

* Very flexible across predictive applications
* Not appropriate for descriptive applications

### Strengths

* Fast to train
* Can make speed/performance tradeoff by tuning parameters
* Flexible across many/few features, many/few data points

### Weaknesses

* Slow to make predictions once trained
* Results of learning are not descriptive/understandable

### Why choose this model?

It can perform acceptably with many features and few data points. It would be interesting to see the speed/performance tradeoff for this data set - since our data set is small and we will predict in small volumes, we can lean towards performance.

### Notes

Changing the number of estimators in the classifier by an order or magnitude (1, 10, 100) appeared to change the F1 score by ~3%.

In [11]:
from sklearn.ensemble import RandomForestClassifier
import math
# scikit-learn recommends max_depth=sqrt(n_features)
# http://scikit-learn.org/stable/modules/ensemble.html#parameters
clf = RandomForestClassifier(
    n_estimators=12, 
    max_depth=int(math.sqrt(n_features))
)
print "========="
print "===RFC==="
print "========="
train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf, X_train[0:100], y_train[0:100], X_test, y_test)
train_predict(clf, X_train[0:200], y_train[0:200], X_test, y_test)
train_predict(clf, X_train[0:300], y_train[0:300], X_test, y_test)

===RFC===
------------------------------------------
Training set size: 300
Training RandomForestClassifier...
Done!
Training time (secs): 0.029
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.882217090069
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.847682119205
------------------------------------------
Training set size: 100
Training RandomForestClassifier...
Done!
Training time (secs): 0.020
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.926174496644
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.84076433121
------------------------------------------
Training set size: 200
Training RandomForestClassifier...
Done!
Training time (secs): 0.023
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.

## K Nearest Neighbours

### Complexity

Computational:

Training (when pre-processing k): O(n*m) where n is the number of data points and m is the number of features

Testing: O(n*m)

Space: O(n*m)

### General Applications
* Economic forecasting [1]
* Breast cancer diagnoses [2]

### Strengths

* Simple and easy to implement
* Nearest Neighbours is used for clustering

### Weaknesses

* Does not 'learn' anything from training data - just uses the entire set of training data for classification
* Slow for large samples
* Changing K can change class labels
* Not robust to noisy data
* Does not give probabilistic output

### Why choose this model?

I'm curious how a type of model more often used for clustering would perform at prediction.

[1] http://www.ijera.com/papers/Vol3_issue5/DI35605610.pdf

[2] http://www.ncbi.nlm.nih.gov/pmc/articles/PMC2243774/


In [12]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
print "========="
print "===KNN==="
print "========="
train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf, X_train[0:100], y_train[0:100], X_test, y_test)
train_predict(clf, X_train[0:200], y_train[0:200], X_test, y_test)
train_predict(clf, X_train[0:300], y_train[0:300], X_test, y_test)

===KNN===
------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.002
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.013
F1 score for training set: 0.849765258216
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.040
F1 score for test set: 0.813793103448
------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.002
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.838709677419
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.802631578947
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.005
F1 score fo

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

### Best Model

K Nearest Neighbours is the model I would recommend. Predictive performance is the deciding factor of model choice, as long as the time and space complexities don't make the model unreasonable to use. Here is a summary of the model performances on the whole dataset:

#### K Nearest Neighbours

* Training time: 0.001 sec
* Prediction time: 0.007 sec
* Training F1 Score: ~0.87
* Testing F1 Score: ~0.80

#### Forest of Randomized Decision Trees

* Training time: 0.011 sec
* Prediction time: 0.010 sec
* Training F1 Score: ~0.89
* Testing F1 Score: ~0.77 - 0.80

#### SVC

* Training time: 0.001 sec
* Prediction time: 0.007 sec
* Training F1 Score: ~0.87
* Testing F1 Score: ~0.78 - 0.8

K Nearest Neighbours performs best at Testing F1 score. Not only is it always higher than the other methods, but it is consistent given the same training/test split- unlike the others which vary in performance every execution on the same training/test data split. KNN also has closer Training and Testing F1 scores - which indicates that it is not overfitting as much as the other models.

KNN has significant disadvantages which don't appear to prevent from it being the best choice. The prediction time will scale quickly with data points. In this case, there were 300 training points, ~100 testing points, and x features. Prediction time on my computer was 0.007 sec = O(x * 300) * 100 which means each prediction will take about 0.00000023 * # training points. If there are 10000 students that took a particular course in the past, and you have to make predictions on that course for 500 new students, it will take ~1.17 sec to perform all the predictions. This appears to be tolerable. If this dataset was a few orders of magnitude larger, KNN would be unreasonable.

Finally, we don't seem to need to understand how the model is making predictions (we don't care about descriptiveness), so KNN does not fail us in that respect.

### How the Model Works

During training, KNN simply stores each data point (the student and all their properties) and the class it belongs to (whether the student passed or failed). When predicting what class a new data point should belong to (whether or not a new student will pass), KNN finds the top k most similar data points (where k is some number we chose beforehand). The class we predict for the new data point is the most commonly found class in the k data points we just found. The way we define how 'similar' any two data points are can be customized, but there are some commonly-used ways to choose from.


### Final Score

~0.80

Surprisingly, GridSearchCV (consistently) picked a set of parameters that performed marginally worse than the default parameters. GridSearchCV must have performed a different split on the training data to test which one performs best (so the training data used to train wasn't the exact same for the grid search vs trial with default  




In [13]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

def performance_metric(y_true, y_predict):
    return f1_score(y_true, y_predict, pos_label='yes')

scoring_function = make_scorer(
    performance_metric,
    greater_is_better=True
)

parameters_to_tune = {
    'n_neighbors': (1, 2, 5, 8, 10, 15, 30, 50),
    'weights': ('uniform', 'distance'),
    'algorithm': ('ball_tree', 'kd_tree', 'brute')
}


best_knn = GridSearchCV(
    KNeighborsClassifier(),
    parameters_to_tune,
    scoring=scoring_function
)
best_knn.fit(X_train, y_train)

train_predict(best_knn.best_estimator_, X_train, y_train, X_test, y_test)

print best_knn.best_estimator_
print best_knn.best_score_


------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.007
F1 score for training set: 0.803493449782
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.828947368421
KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=30, p=2,
           weights='uniform')
0.801677746122
